# Tarea 4

## Ejercicio 1 

En este ejercicio intentaremos comprobar numericamente a través de perturbaciones en la matriz de vectores propios y en los valores propios, si el Jacobiano que utilizamos para la distribución de los valores propios de las matrices pertencientes al ensemble GOE, obtener su determinante es equivalente al determinante de Vandermonde.

In [56]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns


In [57]:
# definimos n
n = 10

#matriz a evaluar
X = np.random.normal(0, 1,(n,n)) 
H=( X + np.transpose(X))/2

#eigenvalores
L, O = np.linalg.eig(H)

#dimension jacobiano
k = int( (n * (n+1))/2 )

#Jacobiano simulado
Jacobiano = np.zeros((k,k))
up = np.triu(np.ones((n,n)), k=1)

#perturbacion
epsilon = 1E-5
#indice para rellenar las columnas de Jacobiano
idx = 0


In [58]:
# calculo
for i in range(n):
    for j in range(i,n):
        #Matriz con perturbaciones
        E = np.zeros((n,n))
        #Solo afectamos los valores i,j 
        E[i,j] = 1
        E[j,i] = 1
        #Perturbamos la matriz
        Hp = H + epsilon*E
        
        #obtenemos sus eigenvalores
        Lp, Op = np.linalg.eig(Hp)
        #la diferencia vs la matriz original en eigenvalores
        dL = (Lp-L)/epsilon
        #la diferencia vs la matriz original en eigenvectores
        OpO = (np.transpose(O)@(Op - O) )/epsilon
        
        #primeros n renglones llenamos con la diferencia de los
        #eigenvalores
        Jacobiano[0:n,idx] = dL
        #despues rellenamos con la parte superior de la diferencia
        #de eigenvectores
        Jacobiano[n:,idx] = OpO[up!=0]
        #vamos cambiando de columna
        idx=idx+1

In [62]:
# Comparacion
print("Determinante Vandermonde")
# el determinante de Vandermonde
print( 1/np.abs(np.linalg.det(np.vander(L))) )

print("")
print("Determinante del Jacobiano construido con perturbaciones numéricas")
#con el jacobiano calculado con diferencias
print ( np.abs(np.linalg.det(Jacobiano) ))


Determinante Vandermonde
2.455436616159924e-16

Determinante del Jacobiano construido con perturbaciones numéricas
2.455440872107091e-16


Dado este resultado podemos observar que si se aproximan en gran manera.

## Ejercicio 2 
Para este ejercicio vamos a verificar si la integrando el Resolvente, aplicando la fórmula del semicírculo, es consistente con la expresión:
$$G_\infty^{av}= z \pm \sqrt{z^2-2}$$


In [1]:
# importamos librerias para las integrales
from math import sqrt
from math import pi
import scipy
from scipy.integrate import quad


In [2]:
#para separar la parte real y la parte imaginaria de la función 
def complex_quadrature(func, a, b):
    def real_func(x):
        return scipy.real(func(x))
    def imag_func(x):
        return scipy.imag(func(x))
    real_integral = quad(real_func, a, b)
    imag_integral = quad(imag_func, a, b)
   # nos regrese de igual forma separado
    return (real_integral[0] + 1j*imag_integral[0])


In [3]:
# Para el resolvente teorico con funcion semicirculo
def resolvente (la, eps):
    #valor de z
    z = la-1j*eps
    
    #donde vamos a integrar
    #definimos limites de integracion
    if   (la > 0 and la < sqrt(2)): 
        limites = np.array([0,sqrt(2)])
    elif (la < 0 and la > -sqrt(2)):
        limites = np.array([-sqrt(2),0 ])
    else: 
        print ("lambda invalida") 
    
    #para la funcion del semicirculo y z
    f1 = lambda x: ( (1/pi) * (2-x**2)**.5 ) / (z-x)
    
    #regresamos la integral evaluada
    return  complex_quadrature(f1, limites[0], limites[1])



In [26]:
def resolvEstimado (la, eps):
    z = la-1j*eps
    #calculamos el resolvente pero con la funcion derivada en clase

    if   (la > 0 and la < sqrt(2)): 
        g = z - (z**2-2)**.5
    elif (la < 0 and la > -sqrt(2)):
        g = z + (z**2-2)**.5
    else: 
        print ("lambda invalida") 
    return g


In [47]:
# valores para la valuación

lambda1 = np.linspace(-sqrt(2)+.001,sqrt(2)-.001, 20)
epsilon1 = np.linspace (.5, 1, 20)

#se guardará información
r1= np.empty(0, dtype=complex)
r2 = np.empty(0, dtype=complex)

for x in range(20): 
    r1 = np.concatenate( (r1, resolvente(lambda1[x],epsilon1[x] )),    axis=None)
    r2 = np.concatenate(( r2, resolvEstimado(lambda1[x],epsilon1[x])), axis=None)


In [48]:
r1 = r1.round(4)
r2 = r2.round(4)

In [49]:
result = pd.DataFrame({'Lambda':lambda1,  'epsilon': epsilon1,
                       'Resolvente Teorico': r1,
                       'Resolvente Estimado': r2,
                       'Error Relativo 1' : np.abs(r1-r2) } )


In [50]:
result.round(4)


,Lambda,epsilon,Resolvente Teorico,Resolvente Estimado,Error Relativo 1
0,-1.4132,0.5000,(-0.4045+0.3553j),(-0.6442+0.4188j),0.2480
1,-1.2645,0.5263,(-0.371+0.4384j),(-0.6258+0.5158j),0.2663
2,-1.1157,0.5526,(-0.3129+0.5114j),(-0.5838+0.6065j),0.2871
3,-0.9669,0.5789,(-0.2363+0.5672j),(-0.5239+0.6846j),0.3106
4,-0.8182,0.6053,(-0.1482+0.602j),(-0.4522+0.7478j),0.3372
5,-0.6694,0.6316,(-0.0549+0.6143j),(-0.3733+0.7962j),0.3667
6,-0.5207,0.6579,(0.0372+0.6036j),(-0.2905+0.8303j),0.3985
7,-0.3719,0.6842,(0.1217+0.5711j),(-0.2062+0.8511j),0.4312
8,-0.2231,0.7105,(0.1915+0.5203j),(-0.1222+0.8596j),0.4621
9,-0.0744,0.7368,(0.2416+0.4581j),(-0.04+0.8565j),0.4879


No se pudo obtener el resultado esperado, dado que quizá la manera de comparar los números complejos no sea la mejor, o no se aplicó bien la metodología pues nuestro error es un poco grande.